In [17]:
from gurobipy import *
from prettytable import PrettyTable
from numpy import array, concatenate,append

EC = Model('Economic Dispatch')
Gen = ['Gen1','Gen2', 'Gen3', 'Gen4', 'Gen5']
GenAt = {Gen[0]:{'a':3 ,'b':20 ,'c':100 , 'Pmin':28 ,'Pmax':206 },
      Gen[1]:{'a':4.05 ,'b':18.07 ,'c':98.87 , 'Pmin':90 ,'Pmax':284 },
      Gen[2]:{'a':4.05 ,'b':15.55 ,'c':104.26 , 'Pmin':68 ,'Pmax':189 },
      Gen[3]:{'a':3.99 ,'b':19.21 ,'c':107.21 , 'Pmin':76 ,'Pmax':266 },  
      Gen[4]:{'a':3.88 ,'b':26.18 ,'c':95.31 , 'Pmin':19 ,'Pmax':53 }  }



Iter =11
Load=[]
TotalLoad=[]
Objective=[]
Var=[]
for c in range(Iter):
    Demand = quicksum(GenAt[i]['Pmin'] for i in Gen) + \
    ((c)/(Iter-1))*quicksum(GenAt[i]['Pmax'] - GenAt[i]['Pmin'] for i in Gen)
    Load.append(Demand)




for k in range(Iter):
   
    P = EC.addVars(Gen,lb = [GenAt[i]['Pmin'] for i in Gen], ub = [GenAt[i]['Pmax'] for i in Gen],name = 'Power')
    GenTotal = quicksum(P[i] for i in Gen)
    GenCost = quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i]+GenAt[i]['a']*P[i]*P[i] for i in Gen)
    EC.setObjective(GenCost, GRB.MINIMIZE)
    target = EC.addConstr( GenTotal == Load[k], name = 'DemandConst')
   
    

    EC.Params.Method=-1
    EC.setParam( 'OutputFlag', False )
    EC.optimize()
    TotalLoad.append(Load[k].getValue())
    v = EC.getVars()
    Obj = EC.objVal
    Objective.append(EC.objVal)
    

    status = EC.status
    if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE \
    or status == GRB.Status.UNBOUNDED:
        EC.computeIIS()
        EC.write("model.ilp")

print("Total Load per period")    
print(TotalLoad)

print( "Objective per period")
print( Objective)

for v in EC.getVars():
        Var.append(v.x)
        
Power = array(Var).reshape(Iter,len(Gen))
print("Power(Period, Generated)")
print(Power)

Parameter Method unchanged
   Value: -1  Min: -1  Max: 5  Default: -1
Total Load per period
[281.0, 352.7, 424.4, 496.1, 567.8, 639.5, 711.2, 782.9, 854.6, 926.3000000000001, 998.0]
Objective per period
[84037.84999999999, 105758.02056686045, 147715.9420639002, 203091.65467032706, 268132.17460228805, 342712.4082488302, 426832.35560995335, 522137.8073304251, 631691.6723880627, 760550.4963675376, 911044.09]
Power(Period, Generated)
[[ 28.   90.   68.   76.   19. ]
 [ 99.7  90.   68.   76.   19. ]
 [171.4  90.   68.   76.   19. ]
 [ 28.   90.   93.1 266.   19. ]
 [ 28.   90.  164.8 266.   19. ]
 [ 28.  284.   68.  240.5  19. ]
 [206.  284.   68.  134.2  19. ]
 [206.  284.   68.  205.9  19. ]
 [ 62.6 284.  189.  266.   53. ]
 [134.3 284.  189.  266.   53. ]
 [206.  284.  189.  266.   53. ]]


In [29]:
TotalLoad = array(TotalLoad,  ndmin=2)
TotalLoad = TotalLoad.T
print(TotalLoad)

[[281. ]
 [352.7]
 [424.4]
 [496.1]
 [567.8]
 [639.5]
 [711.2]
 [782.9]
 [854.6]
 [926.3]
 [998. ]]


In [36]:
Power

array([[ 28. ,  90. ,  68. ,  76. ,  19. ],
       [ 99.7,  90. ,  68. ,  76. ,  19. ],
       [171.4,  90. ,  68. ,  76. ,  19. ],
       [ 28. ,  90. ,  93.1, 266. ,  19. ],
       [ 28. ,  90. , 164.8, 266. ,  19. ],
       [ 28. , 284. ,  68. , 240.5,  19. ],
       [206. , 284. ,  68. , 134.2,  19. ],
       [206. , 284. ,  68. , 205.9,  19. ],
       [ 62.6, 284. , 189. , 266. ,  53. ],
       [134.3, 284. , 189. , 266. ,  53. ],
       [206. , 284. , 189. , 266. ,  53. ]])

In [37]:
TotalLoad

array([[281. ],
       [352.7],
       [424.4],
       [496.1],
       [567.8],
       [639.5],
       [711.2],
       [782.9],
       [854.6],
       [926.3],
       [998. ]])

In [49]:
from numpy import insert,append 
print(append(Power, TotalLoad, axis=1))

[[ 28.   90.   68.   76.   19.  281. ]
 [ 99.7  90.   68.   76.   19.  352.7]
 [171.4  90.   68.   76.   19.  424.4]
 [ 28.   90.   93.1 266.   19.  496.1]
 [ 28.   90.  164.8 266.   19.  567.8]
 [ 28.  284.   68.  240.5  19.  639.5]
 [206.  284.   68.  134.2  19.  711.2]
 [206.  284.   68.  205.9  19.  782.9]
 [ 62.6 284.  189.  266.   53.  854.6]
 [134.3 284.  189.  266.   53.  926.3]
 [206.  284.  189.  266.   53.  998. ]]


array([[ 309. ,  371. ,  349. ,  357. ,  300. ],
       [ 452.4,  442.7,  420.7,  428.7,  371.7],
       [ 595.8,  514.4,  492.4,  500.4,  443.4],
       [ 524.1,  586.1,  589.2,  762.1,  515.1],
       [ 595.8,  657.8,  732.6,  833.8,  586.8],
       [ 667.5,  923.5,  707.5,  880. ,  658.5],
       [ 917.2,  995.2,  779.2,  845.4,  730.2],
       [ 988.9, 1066.9,  850.9,  988.8,  801.9],
       [ 917.2, 1138.6, 1043.6, 1120.6,  907.6],
       [1060.6, 1210.3, 1115.3, 1192.3,  979.3],
       [1204. , 1282. , 1187. , 1264. , 1051. ]])